# 유튜브 인기동영상 데이터

## 데이터 로드

In [1]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/youtube.csv",index_col=0)
df.head()

,title,channelTitle,categoryId,view_count,likes,dislikes,comment_count,channelId,trending_date2
0,[신병] 물자창고,장삐쭈,23,1893473,38249,730,8595,UChbE5OZQ6dRHECsX0tEPEZQ,2021-01-01
1,RAIN(비) - 나로 바꾸자 Switch to me (duet with JYP) MV,RAIN's Official Channel,10,2600864,0,0,20129,UCxXgIeE5hxWxHG6dz9Scg2w,2021-01-01
2,2020년 제야의 종 온라인 타종행사 | 보신각 현장 행사는 진행하지 않습니다.,서울시 · Seoul,29,347049,3564,120,178,UCZUPZW5idAxYp-Asj__lVAA,2021-01-01
3,고기남자의 칠면조 파티,고기남자 MeatMan,26,528458,15372,280,3470,UCT3CumbFIJiW33uq0UI3zlg,2021-01-01
4,골목 3mc를 분노하게 만든 마음고생이 심했을 공릉 백반집 사장님의 푸념?! [예능...,스브스밥집,24,494904,3918,111,3142,UCdWgRSfttvDucq4ApcCg5Mw,2021-01-01


## 인기동영상 제작횟수가 많은 채널 상위 10개명을 출력(날짜 기준, 중복 포함)

In [2]:
print(df.loc[df.channelId.isin(list(df.groupby('channelId').size().sort_values(ascending=False).keys()[:10]))].channelTitle.unique())

['장삐쭈' '총몇명' '파뿌리' '짤툰' '런닝맨 - 스브스 공식 채널' '엠뚜루마뚜루 : MBC 공식 종합 채널' 'SPOTV'
 '채널 십오야' '이과장' 'BANGTANTV']


## 논란으로 인기동영상이 된 케이스 확인, dislikes 수가 like 수보다 높은 동영상을 제작한 채널을 모두 출력

In [3]:
print(df.loc[df.dislikes > df.likes].channelTitle.unique())

['핫도그TV' 'ASMR 애정TV' '하얀트리HayanTree' '양팡 YangPang' '철구형 (CHULTUBE)'
 '왜냐맨하우스' '(MUTUBE)와꾸대장봉준' '오메킴TV' '육지담' 'MapleStory_KR'
 'ROAD FIGHTING CHAMPIONSHIP' '사나이 김기훈' '나혼자산다 STUDIO' 'Gen.G esports']


## 채널명을 한번이라도 바꾼 채널의 갯수

In [4]:
change = df[['channelId','channelTitle']].drop_duplicates().channelId.value_counts()
print(len(change[change>1]))

71


## 일요일에 인기있었던 영상들 중 가장 많은 영상 종류

In [5]:
# df['weekends'] = df['trending_date2'].dt.weekends
df['trending_date2'] = pd.to_datetime(df['trending_date2'])
df['weekday'] = df['trending_date2'].dt.day_name()
print(df.loc[df.weekday=='Sunday'].categoryId.value_counts().index[0])

24


## 각 요일별 인기 영상들의 categoryID는 각각 몇개 씩인지 하나의 데이터 프레임으로 표현하라

In [6]:
print(pd.pivot_table(df,index='categoryId',columns='weekday',aggfunc='size'))

weekday     Friday  Monday  Saturday  Sunday  Thursday  Tuesday  Wednesday
categoryId                                                                
1              243     263       255     274       246      257        234
2              120     105       119      99       128      119        129
10             833     837       776     830       890      894        917
15             187     215       198     217       207      208        207
17             633     668       592     636       682      708        706
19              90      92        87      91        92       89         85
20             283     298       296     289       282      285        291
22            1288    1373      1289    1337      1341     1375       1333
23             568     594       570     556       560      569        566
24            2976    3148      3066    3096      2954     3084       3090
25             444     453       422     437       470      452        468
26             369     37

## viewcount 대비 댓글 수가 가장 높은 영상을 확인

In [11]:
quest = df.loc[df.view_count != 0]
quest['feedback'] = quest['comment_count']/quest['view_count'].dropna()

quest.sort_values('feedback',ascending =False).iloc[0]['title']

/tmp/ipykernel_762/1598414501.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quest['feedback'] = quest['comment_count']/quest['view_count'].dropna()


'60분 동안 댓글이 달리지 않으면, 영상이 삭제됩니다. (챌린지)'

## viewcount 대비 댓글 수가 가장 높은 영상을 확인

In [22]:
quest = df.loc[df.view_count !=0]
quest['feedback'] = (quest.comment_count/quest.view_count)
quest = quest.loc[quest.feedback != 0]
quest.sort_values('feedback').iloc[0]['title']

/tmp/ipykernel_762/2899900271.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quest['feedback'] = (quest.comment_count/quest.view_count)


'Join the BTS #PermissiontoDance Challenge only on YouTube #Shorts'

## 좋아요 수 대비 싫아요 비율이 가장 낮은 영상 확인

In [23]:
quest = df.loc[(df.likes != 0) & (df.dislikes != 0)]
quest['ratio']=df.dislikes/df.likes
quest.sort_values('ratio').iloc[0]['title']

/tmp/ipykernel_762/2567045718.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quest['ratio']=df.dislikes/df.likes


'[줌터뷰] *최초공개* 사부작즈🐰🐶의 비공식 이름은 아이라인즈? 꿀조합 티키타카 가득한 NCT 127 도영&정우의 줌터뷰'

## 가장 많은 트렌드 영상을 제작한 채널의 이름

In [24]:
print(df.loc[df.channelId == df.channelId.value_counts().index[0]].channelTitle.unique()[0])

짤툰


## 20회 이상 인기동영상 리스트에 포함된 동영상의 숫자

In [25]:
print((df[['title','channelId']].value_counts() >= 20).sum())


40


# 유튜브 공범컨텐츠 동영상 데이터

## 데이터 로드

In [26]:
import pandas as pd
channel =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/channelInfo.csv')
video =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/videoInfo.csv')

## 각 데이터의 'ct' 컬럼을 시간으로 인식할 수 있게 datatype을 변경하고 video 데이터의 videoname의 각 value 마다 몇 개의 데이터씩 가지고 있는지 확인

In [32]:
channel['ct'] = pd.to_datetime(channel['ct'])
video['ct'] = pd.to_datetime(video['ct'])
print(video['videoname'].value_counts())

videoname
공범 EP1    3492
공범 EP2    3204
공범 EP3    2568
공범 EP4    2280
공범 EP5    1562
공범 EP6    1274
공범 EP7     555
공범 EP8     266
Name: count, dtype: int64


## 수집된 각 vide의 가장 최신화된 날짜의 viewcount 값 출력

In [43]:
print(video[['viewcnt','videoname','ct']].sort_values(['videoname','ct']).drop_duplicates('videoname',keep='last').reset_index(drop=True))

   viewcnt videoname                  ct
0  3180532    공범 EP1 2021-11-01 15:30:03
1  2199328    공범 EP2 2021-11-01 15:30:03
2  1671294    공범 EP3 2021-11-01 15:30:03
3  1818493    공범 EP4 2021-11-01 15:30:03
4  1503435    공범 EP5 2021-11-01 15:30:04
5  1750222    공범 EP6 2021-11-01 15:30:04
6  1630200    공범 EP7 2021-11-01 15:30:05
7  1289088    공범 EP8 2021-11-01 15:30:05


## channel 데이터 중 2021-10-03일 이후 각 채널의 처음 기록 됐던 구독자 수 출력

In [51]:
quest = channel.loc[channel.ct >= '2021-10-03'].sort_values(['channelid','ct']).drop_duplicates('channelid',keep='first')
print(quest[['channelname','subcnt']].reset_index(drop=True))

        channelname   subcnt
0            논리왕 전기   922000
1             야전삽짱재   257000
2           형사!탐정되다    14900
3       릴펄 Lilpearl    10100
4               조나단   215000
5        김농밀의 농밀한 삶     7520
6     Balming Tiger    54300
7               꽈뚜룹  1330000
8   와글와글 WagleWagle    55000
9        츄정ChuJeong   322000
10    곽토리 kwak tori   471000


## 각 채널의 2021-10-03 03:00:00 ~ 2021-11-01 15:00:00 까지 구독자수의 증가량

In [74]:
before = channel.loc[channel.ct >= '2021-10-03 03:00:00'].sort_values(['channelid','ct']).drop_duplicates('channelid',keep='first')
before = before.rename(columns={'subcnt':'before'})
after = channel.loc[channel.ct >= '2021-11-01 15:00:00'].sort_values(['channelid','ct']).drop_duplicates('channelid',keep='last')
after = after.rename(columns={'subcnt':'after'})
after
total = pd.merge(before[['channelid','channelname','before']],after[['channelid','channelname','after']])
total['del'] = total['after'] - total['before']
print(total[['channelname','del']])

        channelname    del
0            논리왕 전기 -11000
1             야전삽짱재  11000
2           형사!탐정되다  10300
3       릴펄 Lilpearl  11000
4               조나단  12000
5        김농밀의 농밀한 삶   1540
6     Balming Tiger   2500
7               꽈뚜룹  70000
8   와글와글 WagleWagle      0
9        츄정ChuJeong   1000
10    곽토리 kwak tori  -2000


## 각 비디오는 10분 간격으로 구독자 수, 좋아요, 싫어요 수, 댓글 수 가 수집됨. 공범 EP1의 비디오 정보 데이터 중 수집간격이 5분 이하, 20분 이상인 데이터 구간(해당 시점 전, 후)의 시각을 모두 출력

In [117]:
part1 = video.loc[video['videoname'] == ' 공범 EP1'].sort_values('ct')
part1.loc[(part1.ct.diff(1) <= '0 days 00:5:00')|(part1.ct.diff(1) >= '0 days 00:20:00')]
print(part1.loc[[720,721,722,723,1635,1636,1637],:])

          videopk  viewcnt  likecnt  dislikecnt  favoritecnt  cmcnt  \
720   c5JQp6xafqc  2153676    34104         830            0   7120   
721   c5JQp6xafqc  2228250    34559         849            0   7191   
722   c5JQp6xafqc  2228250    34559         849            0   7191   
723   c5JQp6xafqc  2228669    34566         849            0   7194   
1635  c5JQp6xafqc  2706691    37893         987            0   7410   
1636  c5JQp6xafqc  2707933    37901         988            0   7410   
1637  c5JQp6xafqc  2708448    37903         988            0   7410   

                      ct videoname  
720  2021-10-12 19:20:03    공범 EP1  
721  2021-10-13 09:41:37    공범 EP1  
722  2021-10-13 09:41:37    공범 EP1  
723  2021-10-13 09:50:03    공범 EP1  
1635 2021-10-19 17:50:02    공범 EP1  
1636 2021-10-19 18:20:03    공범 EP1  
1637 2021-10-19 18:30:03    공범 EP1  


## 각 에피소드의 시작날짜(년-월-일)을 에피소드 이름과 묶어 데이터 프레임으로 만들고 출력

In [133]:
video['date'] = video['ct'].dt.date
result = video.sort_values(['videoname','date']).drop_duplicates('videoname',keep='first')
print(result[['videoname','date']])


      videoname        date
10       공범 EP1  2021-10-07
3496     공범 EP2  2021-10-09
6696     공범 EP3  2021-10-14
9264     공범 EP4  2021-10-16
11544    공범 EP5  2021-10-21
13661    공범 EP6  2021-10-23
13106    공범 EP7  2021-10-28
14935    공범 EP8  2021-10-30


## video 정보의 가장 최근 데이터들에서 각 에피소드의 싫어요/좋아요 비율을 ratio 커럼으로 만들고 videoname,ratio로 구성된 데이터 프레임을 ratio 오름차순으로 정렬

In [142]:
result = video.sort_values(['videoname','ct']).drop_duplicates('videoname',keep='last')
result['ratio'] = result.dislikecnt/result.likecnt
print(result[['videoname','ratio']].sort_values('ratio'))

      videoname     ratio
13660    공범 EP7  0.013636
14934    공범 EP6  0.015881
11543    공범 EP4  0.017971
13105    공범 EP5  0.021128
9263     공범 EP3  0.022360
3491     공범 EP1  0.027118
6695     공범 EP2  0.029844
15200    공범 EP8  0.141357


## 2021-11-01 00:00:00 ~ 15:00:00까지 각 에피소드별 viewcnt의 증가량을 데이터 프레임으로 만드시오

In [153]:
before = video.loc[(video.ct >= '2021-11-01 00:00:00'),['videoname','viewcnt','ct']].sort_values(['videoname','ct']).drop_duplicates('videoname',keep='first')
before = before.rename(columns={'viewcnt':'before'})
after = video.loc[(video.ct <= '2021-11-01 15:00:00'),['videoname','viewcnt','ct']].sort_values(['videoname','ct']).drop_duplicates('videoname',keep='last')
after = after.rename(columns={'viewcnt':'after'})
total = pd.merge(before[['videoname','before']],after[['videoname','after']])
total['viewcnt'] = total['after']-total['before']
print(total[['videoname','viewcnt']])

  videoname  viewcnt
0    공범 EP1    13298
1    공범 EP2    10300
2    공범 EP3     9927
3    공범 EP4     9824
4    공범 EP5    10824
5    공범 EP6    14141
6    공범 EP7    26949
7    공범 EP8    89147


## video 데이터 중에서 중복되는 데이터의 시간대와 videoname

In [174]:
answer = video[video.index.isin(set(video.index) - set(video.drop_duplicates().index))]
print(answer[['videoname','ct']])

     videoname                  ct
722     공범 EP1 2021-10-13 09:41:37
3927    공범 EP2 2021-10-13 09:41:37


# 월드컵 출전선수 골기록 데이터

## 데이터 로드

In [410]:
import pandas as pd

df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/worldcup/worldcupgoals.csv')
df.head()

,Player,Goals,Years,Country
0,Miroslav Klose,16,2002-2006-2010-2014,Germany
1,Ronaldo,15,1998-2002-2006,Brazil
2,Gerd Muller,14,1970-1974,Germany
3,Just Fontaine,13,1958,France
4,Pele,12,1958-1962-1966-1970,Brazil


## 주어진 전체 기간의 각 나라별 골득점수 상위 5개 국가와 그 득점수 출력

In [411]:
print(df[['Country','Goals']].groupby('Country').sum().sort_values('Goals',ascending=False).head(5))


           Goals
Country         
Brazil       228
Germany      226
Argentina    135
Italy        123
France       115


## 골득점을 한 선수가 많은 상위 5개 국가와 그 선수 숫자를 데이터 프레임 형식으로 출력

In [412]:
print(df[['Country','Player']].groupby('Country').size().sort_values(ascending=False).head(5))

Country
Brazil       81
Germany      78
Italy        60
Argentina    59
France       58
dtype: int64


## 년도 표기가 4자리 숫자로 안된 케이스가 몇 건인지 출력

In [413]:
df['Years_list'] = df['Years'].str.split('-')
def check(row):
    answer = False
    for i in row:
        if len(i) != 4:
            answer = True
    return answer
df['Check'] = df['Years_list'].apply(check)
print(len(df.loc[df['Check'] == True]))

45


## 년도 표기가 4자리 숫자로 되어있는 케이스만 추출

In [414]:
df2 = df.loc[df['Check'] == False]
print(len(df2))

1250


## 월드컵 출전횟수를 나타내는 'LenCup' 컬럼을 추가하고 4회 출전한 선수의 숫자 출력

In [415]:
df2['LenCup'] = df.Years_list.map(lambda x : len(x))
print(len(df2.loc[df2.LenCup == 4]))

16


/tmp/ipykernel_762/715742222.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['LenCup'] = df.Years_list.map(lambda x : len(x))


## Yugoslavia 국가의 월드컵 출전횟수가 2회인 선수들의 숫자

In [416]:
print(len(df2[(df2.Country == 'Yugoslavia') & (df2.LenCup == 2)]))

7


## 2002년도에 출전한 전체 선수 

In [417]:
print(len(df2.loc[df2.Years.str.contains('2002')]))

156


## 이름에 'carlos' 단어가 들어가는 선수의 숫자

In [418]:
print(len(df2.loc[df.Player.str.lower().str.contains('carlos')]))

13


## 월드컵 출전 횟수가 1회뿐인 선수들 중에서 가장 많은 득점을 올렸던 선수

In [419]:
print(df2.loc[df2.LenCup == 1].sort_values('Goals',ascending=False)['Player'].values[0])

Just Fontaine


## 월드컵 출전횟수가 1회 뿐인 선수들이 가장 많은 국가

In [420]:
print(df2.loc[df2.LenCup == 1].groupby('Country').size().sort_values(ascending=False).index[0])

Brazil


# 서울시 따릉이 이용정보 데이터

## 데이터 로드

In [421]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bicycle/seoul_bi.csv')
df.head()

,대여일자,대여시간,대여소번호,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리,사용시간
0,2021-06-01,0,3541,정기권,F,~10대,1,0.00,0.00,0.00,8
1,2021-06-01,0,765,정기권,F,~10대,1,27.21,0.35,1526.81,19
2,2021-06-01,0,2637,정기권,F,~10대,1,41.40,0.37,1608.56,18
3,2021-06-01,0,2919,정기권,F,~10대,1,0.00,0.00,0.00,75
4,2021-06-01,0,549,정기권,F,~10대,1,13.04,0.17,731.55,6


## 대여일자별 데이터의 수를 데이터프레임으로 출력하고, 가장 많은 데이터가 있는 날짜 출력

In [422]:
print(df.groupby('대여일자').size())
print(df.groupby('대여일자').size().sort_values(ascending= False).index[0])

대여일자
2021-06-01    48081
2021-06-02    56100
2021-06-03    18985
2021-06-04    56788
2021-06-05    52730
2021-06-06    47787
2021-06-07    52623
2021-06-08    55623
2021-06-09    54507
2021-06-10    36992
2021-06-11    52089
2021-06-12    48569
2021-06-13    43131
2021-06-14    54945
dtype: int64
2021-06-04


## 각 일자의 요일을 표기하고 day_name 칼럼을 추가하고 이를 이용하여 각 요일별 이용 회숫의 총합을 데이터 프레임으로 출력

In [423]:
df['대여일자'] = pd.to_datetime(df['대여일자'])
df['day_name'] = df['대여일자'].dt.day_name()
print(df.groupby('day_name').size())

day_name
Friday       108877
Monday       107568
Saturday     101299
Sunday        90918
Thursday      55977
Tuesday      103704
Wednesday    110607
dtype: int64


## 각 요일별 가장 많이 이용한 대여소의 이용횟수와 대여소 번호를 데이터 프레임으로 출력

In [424]:
df.groupby(['day_name','대여소번호']).size().sort_values(ascending=False).reset_index().drop_duplicates('day_name',keep='first')

,day_name,대여소번호,0
0,Saturday,502,378
1,Sunday,502,372
8,Wednesday,502,282
9,Friday,502,277
10,Tuesday,502,267
19,Monday,502,242
298,Thursday,2715,137


## 나이대별 대여구분 코드의 (일일권/전체횟수) 비율을 구한 후 가장 높은 비율을 가지는 나이대 확인/일일권의 경우 일일권과 일일권(비회원)을 모두 포함

In [425]:
print((df.loc[df['대여구분코드'].str.contains('일일권')].groupby('연령대코드').size() /df.groupby('연령대코드').size()))
print((df.loc[df['대여구분코드'].str.contains('일일권')].groupby('연령대코드').size() /df.groupby('연령대코드').size()).sort_values().index[-1])

연령대코드
20대     0.239913
30대     0.166702
40대     0.110585
50대     0.065414
60대     0.042462
70대~    0.048962
~10대    0.312525
dtype: float64
~10대


## 연령대별 평균 이동거리

In [426]:
print(df.groupby('연령대코드').이동거리.mean())

연령대코드
20대     3211.890552
30대     3341.443859
40대     3514.857416
50대     3593.668100
60대     3538.145737
70대~    3085.039641
~10대    2634.426279
Name: 이동거리, dtype: float64


## 연령대가 20대인 데이터를 추출하고, 이동거리값이 추출한 데이터의 이동거리값의 평균 이상인 데이터를 추출, 최종 추출된 데이터를 대여일자, 대여소 번호 순서로 내림차순 정렬 후 1행부터 200행까지의 탄소량의 평균을 소숫점 3째 자리까지 구하기

In [427]:
answer = df.loc[df.연령대코드 == '20대'].loc[df.이동거리 >= df.이동거리.mean()].sort_values(['대여일자','대여소번호'],ascending=False).reset_index(drop=True)[:200]
answer['탄소량'] = answer['탄소량'].astype(float)
print(round(answer['탄소량'].mean(),3))

1.643


## 6월 7일 ~10의 이용건수의 중앙값

In [428]:
df.loc[(df.대여일자 == '2021-06-07')&(df.연령대코드 == '~10대')].이용건수.median()

1.0

## 평일 출근 시간대의 대여소별 이용 횟수 추출/ 대여시간별 이용횟수의 상위 3개 대여소와 이용횟수

In [429]:
result = df.loc[(~df.day_name.isin(['Saturday','Sunday']))&(df.대여시간.isin([6,7,8]))].groupby(['대여시간','대여소번호']).size().to_frame('이용 횟수')
print(result.sort_values(['대여시간','이용 횟수'],ascending=False).groupby('대여시간').head(3))

            이용 횟수
대여시간 대여소번호       
8    2701     119
     646      115
     1152      92
7    259      104
     230       77
     726       77
6    2744      45
     1125      40
     1028      36


## 이동거리 평균 이상의 이동거리 값을 가지는 데이터의 이동거리 표본표준편차

In [430]:
import numpy as np
print(df.loc[df.이동거리 >= df.이동거리.mean()].이동거리.std())

5092.139707505356


## 남성와 여성의 이동거리값의 평균값

In [432]:
def sex(row):
    if row == 'F' or row == 'f':
        answer = '여성'
    else:
        answer = '남성'
    return answer
df['sex'] = df['성별'].apply(sex)
df
df.groupby('sex').이동거리.mean()

sex
남성    3209.110871
여성    3468.575025
Name: 이동거리, dtype: float64

# 전세계 행복도 지표 데이터

## 데이터 로드

In [433]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/happy2/happiness.csv',encoding='utf-8')
df.head()

,행복랭킹,나라명,점수,상대GDP,사회적지원,행복기대치,선택의 자유도,관대함,부패에 대한인식,년도
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393,2019
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410,2019
2,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341,2019
3,4,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118,2019
4,5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298,2019


## 데이터는 2018년도와 2019년도의 전세계 행복 지수를 표현하고, 각 년도의 행복랭킹 50위 이내의 나라들의 각각의 행복 점수 평균을 데이터 프레임으로 표시

In [446]:
answer = df.loc[(df.년도 == 2018)|(df.년도 == 2019)].sort_values(['년도','점수'],ascending=False).groupby('년도').head(50)
print(answer.groupby('년도').점수.mean().to_frame())

           점수
년도           
2018  6.64678
2019  6.67002


## 2018년도 데이터들만 추출하여 행복점수와 부패에 대한 인식에 대한 상관계수를 구하기

In [457]:
print(df.loc[df.년도 == 2018][['점수','부패에 대한인식']].corr().iloc[0,1])

0.40529152271510027


## 2018년도와 2019년도의 행복랭킹이 변화하지 않은 나라명의 수

In [484]:
df2018 = df.loc[df.년도 == 2018]
df2018 = df2018.rename(columns={'년도':'2018','행복랭킹':'2018 점수'})
df2019 = df.loc[df.년도 == 2019]
df2019 = df2019.rename(columns={'년도':'2019','행복랭킹':'2019 점수'})
df2019 =df2019[['나라명','2019 점수']]
df2018 = df2018[['나라명','2018 점수']]
total = pd.merge(df2018,df2019)
total['diff'] = total['2019 점수'] - total['2018 점수']
# total.loc[total.diff == 0]
print(len(total.loc[total['diff']==0]))

15


## 2019년도 데이터들만 추출하여 각 변수간 상관계수를 구하고 내림차순으로 정렬한 후 상위 5개를 데이터 프레임으로 출력, 컬렴명 v1, v2, corr 표시

In [501]:
df2019 = df.loc[df.년도 == 2019]
result = df2019[['상대GDP','사회적지원','행복기대치','선택의 자유도','관대함','부패에 대한인식']].corr().unstack().reset_index().drop_duplicates()
result = result.rename(columns={'level_0':'v1','level_1':'v2',0:'corr'})
print(result.loc[result['corr'] != 1].sort_values('corr',ascending=False).reset_index(drop=True).head(5))

      v1     v2      corr
0  상대GDP  행복기대치  0.835462
1  행복기대치  상대GDP  0.835462
2  상대GDP  사회적지원  0.754906
3  사회적지원  상대GDP  0.754906
4  사회적지원  행복기대치  0.719009


## 각 년도별 하위 행복점수의 하위 5개 국가의 평균 행복점수

In [511]:
result = df.sort_values(['년도','점수']).groupby('년도').head(5)
print(result.groupby('년도').점수.mean())

년도
2018    3.1800
2019    3.1408
Name: 점수, dtype: float64


## 각 년도의 부패에 대한 인식을 내림차순 정렬했을 때 상위 20개 국가의 부패에 대한 인식의 평균

In [515]:
result = df.sort_values(['년도','부패에 대한인식'],ascending=False).groupby('년도').head(20)
print(result.groupby('년도')['부패에 대한인식'].mean())

년도
2018    0.3267
2019    0.3201
Name: 부패에 대한인식, dtype: float64


## 2018년도 행복랭킹 50위 이내에 포함됐다가 2019년 50위 밖으로 밀려난 국가의 숫자

In [519]:
df2018 = df.loc[(df.행복랭킹 <= 50)&(df.년도 == 2018)]
df2019 = df.loc[(df.행복랭킹 >= 50)&(df.년도 == 2019)]
total = pd.merge(df2018['나라명'],df2019['나라명'])
print(len(total))

4


## 2018년, 2019년 모두 기록이 있는 나라들 중 년도별 행복점수가 가장 증가한 나라와 그 증가 수치

In [529]:
df2018 = df.loc[df.년도 == 2018]
df2019 = df.loc[df.년도 == 2019]
df2018 = df2018[['나라명','점수']]
df2018 = df2018.rename(columns={'점수':'2018점수'})
df2019 = df2019[['나라명','점수']]
df2019 = df2019.rename(columns={'점수':'2019점수'})
total = pd.merge(df2018,df2019)
total['diff'] = total['2019점수'] - total['2018점수']
total.sort_values('diff',ascending=False).head(1)

,나라명,2018점수,2019점수,diff
151,Burundi,2.905,3.775,0.87


# 지역구 에너지 소비량 데이터

## 데이터 로드

In [530]:
import pandas as pd
df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/consum/Tetuan%20City%20power%20consumption.csv')
df.head()

,DateTime,Temperature,Humidity,Wind Speed,general diffuse flows,diffuse flows,Zone 1 Power Consumption,Zone 2 Power Consumption,Zone 3 Power Consumption
0,1/1/2017 0:00,6.559,73.8,0.083,0.051,0.119,34055.69620,16128.87538,20240.96386
1,1/1/2017 0:10,6.414,74.5,0.083,0.070,0.085,29814.68354,19375.07599,20131.08434
2,1/1/2017 0:20,6.313,74.5,0.080,0.062,0.100,29128.10127,19006.68693,19668.43373
3,1/1/2017 0:30,6.121,75.0,0.083,0.091,0.096,28228.86076,18361.09422,18899.27711
4,1/1/2017 0:40,5.921,75.7,0.081,0.048,0.085,27335.69620,17872.34043,18442.40964


## dataframe 컬럼을 통해 각 월별로 몇개의 데이터가 있는지 데이터 프레임으로 출력

In [536]:
df['month'] = pd.to_datetime(df.DateTime).dt.month
print(df.groupby('month').size())

month
1     4464
2     4032
3     4464
4     4320
5     4464
6     4320
7     4464
8     4464
9     4320
10    4464
11    4320
12    4320
dtype: int64


## 3월달의 각 시간대별 온도 평균들 중 가장 낮은 시간대의 온도 출력

In [549]:
df['hour'] = pd.to_datetime(df.DateTime).dt.hour
print(df.loc[df.month == 3].groupby('hour').Temperature.mean().sort_values().head(1))

hour
7    11.506613
Name: Temperature, dtype: float64


## 3월달의 각 시간대별 온도의 평균들 중 가장 높은 시간대의 온도 출력

In [551]:
df['hour'] = pd.to_datetime(df.DateTime).dt.hour
print(df.loc[df.month == 3].groupby('hour').Temperature.mean().sort_values(ascending=False).head(1))

hour
15    18.393602
Name: Temperature, dtype: float64


## Zone 1 Power Consumption 컬럼의 value 값의 크기가 Zone 2 Power Consumption 컬럼의 value값의 크기보다 큰 데이터들의 Humidity의 평균

In [563]:
df['diff'] = df['Zone 1 Power Consumption'] - df['Zone 2  Power Consumption']
print(df.loc[df['diff'] > 0].Humidity.mean())

68.23624448055052


## 각 zone의 에너지 소비량의 상관관계를 구해서 데이터 프레임으로 표기

In [570]:
df[['Zone 1 Power Consumption','Zone 2  Power Consumption','Zone 3  Power Consumption']].corr()

,Zone 1 Power Consumption,Zone 2 Power Consumption,Zone 3 Power Consumption
Zone 1 Power Consumption,1.000000,0.834519,0.750733
Zone 2 Power Consumption,0.834519,1.000000,0.570932
Zone 3 Power Consumption,0.750733,0.570932,1.000000


## Temperature의 값이 10미만의 경우 A, 10이상 20미만의 경우 B, 20 이상 30 미만의 경우 C, 그 외의 경우 D라고 할 때 각 단계의 데이터 숫자

In [575]:
def temperature(row):
    if row < 10:
        answer = "A"
    elif row >= 10 and row < 20:
        answer = 'B'
    elif row >= 20 and row < 30:
        answer = 'C'
    else:
        answer = 'D'
    return answer
df['grade'] = df['Temperature'].apply(temperature)
print(df.groupby('grade').size())

grade
A     2874
B    26993
C    21105
D     1444
dtype: int64


## 6월 데이터 중 12시의 Temperature의 표준편차

In [578]:
df.loc[(df['month'] == 6)&(df['hour'] == 12)].Temperature.std()

2.0499417827951025

## 6월 데이터 중 12시의 Temperature의 분산

In [580]:
print(df.loc[(df['month'] == 6)&(df['hour'] == 12)].Temperature.var())

4.202261312849163


## Temperature의 평균 이상의 Temperature의 값을 가지는 데이터를 Temperature를 기준으로 정렬 했을 때 4번째 행의 Humidity

In [587]:
print(df.loc[df['Temperature']>= df['Temperature'].mean()].sort_values('Temperature').reset_index(drop=True).loc[3,'Humidity'])

87.9


## Temperature의 중간값 이상의 Temperature의 값을 가지는 데이터를 Temperature를 기준으로 정렬했을 때 4번째 행의 Humidity

In [589]:
print(df.loc[df['Temperature']>= df['Temperature'].median()].sort_values('Temperature').reset_index(drop=True).loc[3,'Humidity'])

80.3


# 포켓몬 정보 데이터

## 데이터 로드

In [590]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/pok/Pokemon.csv')
df.head()


,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


## 전설포켓몬과 그렇지 않은 포켓몬들의 HP 평균 차이

In [594]:
print(df.loc[df.Legendary == True].HP.mean() - df.loc[df.Legendary == False].HP.mean())

25.55614861329147


## 가장 많은 부속성 종류

In [612]:
print(df['Type 2'].value_counts().index[0])

Flying


## 가장 많은 Type 1의 종의 평균 Attack을 평균 Defense로 나눈 값

In [618]:
print(df.loc[df['Type 1'] == df['Type 1'].value_counts().index[0]].Attack.mean()/df.loc[df['Type 1'] == df['Type 1'].value_counts().index[0]].Defense.mean())

1.0165238678090576


## 포켓몬 세대 중 가장 많은 Legendary를 보유한 세대

In [622]:
print(df.loc[df.Legendary == True].Generation.value_counts().index[0])

3


## Hp, Attack, Defense, Sp.Atk, Sp.Def, Speed 간의 상관계수 중 가장 절댓값이 큰 두 변수와 그 값

In [641]:
result = df[['HP','Attack','Defense','Sp. Atk','Sp. Def','Speed']].corr().unstack().reset_index().drop_duplicates()
result[0] = abs(result[0])
print(result.loc[result[0 ]!= 1].sort_values(0,ascending=False).iloc[0])

level_0     Sp. Def
level_1     Defense
0          0.510747
Name: 26, dtype: object


## 각 Generation의 Attack으로 오름차순 정렬 시 상위 3개 데이터들(18개)의 Attack의 전체 평균

In [648]:
result = df.sort_values(['Generation','Attack'])
print(result.groupby('Generation').Attack.head(3).mean())

19.5


## 각 Geneartion의 Attack으로 내림차순 정렬 시 상위 5개 데이터들(30개)의 Attack의 전체 평균

In [649]:
result = df.sort_values(['Generation','Attack'],ascending=False)
print(result.groupby('Generation').Attack.head(5).mean())

157.23333333333332


## 가장 흔히 발견되는 (Type1,Type2)의 쌍

In [657]:
print(df[['Type 1','Type 2']].value_counts().head(1))

Type 1  Type 2
Normal  Flying    24
Name: count, dtype: int64


## 한번씩만 존재하는 (Type1,Type2)의 쌍을 각 세대별로 가지고 있는 갯수

In [736]:
result = df[['Type 1','Type 2']].value_counts()
result = result[result==1]
result_list = list(result.index)
result = result.reset_index()
def generation(row):
    type1 = row['Type 1']
    type2 = row['Type 2']
    answer = df.loc[(df['Type 1'] == type1)&(df['Type 2'] == type2),'Generation'].values[0]
    return answer
result['generation'] = result.apply(generation,axis=1)
print(result.generation.value_counts())

generation
4    13
6     9
5     7
3     5
2     4
1     1
Name: count, dtype: int64


# 대한민국 체력장 데이터

## 데이터 로드

In [737]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/body/body.csv')
df.head()

,측정나이,측정회원성별,신장 : cm,체중 : kg,체지방율 : %,이완기혈압(최저) : mmHg,수축기혈압(최고) : mmHg,악력D : kg,앉아윗몸앞으로굽히기 : cm,교차윗몸일으키기 : 회,제자리 멀리뛰기 : cm,등급
0,59.0,M,175.0,70.6,19.2,91.0,150.0,40.6,12.2,30.0,179.0,C
1,40.0,F,161.4,45.8,21.7,62.0,119.0,22.1,16.0,32.0,165.0,B
2,27.0,M,176.6,84.8,19.2,87.0,153.0,45.9,13.2,61.0,216.0,B
3,38.0,M,167.6,76.7,24.7,63.0,132.0,43.6,16.0,45.0,231.0,A
4,21.0,M,165.2,66.2,21.5,83.0,106.0,33.5,10.6,46.0,198.0,C


## 전체 데이터의 수축기혈압(최고) - 이완기혈압(최저)의 평균

In [742]:
print((df['수축기혈압(최고) : mmHg'] - df['이완기혈압(최저) : mmHg']).mean())

52.19539414750672


## 50-59세의 신장 평균

In [745]:
print(df.loc[(df['측정나이'] < 60) & (df['측정나이'] >= 50)]['신장 : cm'].mean())

164.07490107405314


## 연령대 (20-29: 20대...)별 인원수

In [748]:
df['연령대'] = df['측정나이'].map(lambda x : int(x//10)*10)
df['연령대'].value_counts()

연령대
20    5831
30    2660
40    1801
50    1769
60    1335
Name: count, dtype: int64

## 연령대(20-20: 20대...)별 등급의 숫자를 데이터 프레임으로 표현

In [752]:
print(df.groupby(['연령대','등급']).size().reset_index())

    연령대 등급     0
0    20  A  1585
1    20  B  1443
2    20  C  1455
3    20  D  1348
4    30  A   743
5    30  B   697
6    30  C   626
7    30  D   594
8    40  A   386
9    40  B   428
10   40  C   455
11   40  D   532
12   50  A   321
13   50  B   410
14   50  C   474
15   50  D   564
16   60  A   314
17   60  B   371
18   60  C   339
19   60  D   311


## 남성 중 A 등급과 D 등급의 체지방률 평균의 차이

In [761]:
print(df.loc[(df.측정회원성별 =='M')&(df.등급 =='D')]['체지방율 : %'].mean() - df.loc[(df.측정회원성별 =='M')&(df.등급 =='A')]['체지방율 : %'].mean())

7.932086486137521


## 여성 중 A 등급과 D 등급의 체중의 평균의 차이

In [764]:
print(df.loc[(df.측정회원성별 =='F')&(df.등급 =='D')]['체중 : kg'].mean() - df.loc[(df.측정회원성별 =='F')&(df.등급 =='A')]['체중 : kg'].mean())

5.176211590296504


## 남성의 BMI 평균

In [768]:
df['BMI'] = df['체중 : kg']/(df['신장 : cm']/100)**2
print(df.loc[df['측정회원성별'] == 'M'].BMI.mean())

24.46134409819308


## BMI 보다 체지방율이 높은 사람들의 체중 평균

In [771]:
print(df.loc[df['체지방율 : %'] > df['BMI']]['체중 : kg'].mean())

61.7408806392542


## 남성과 여성의 악력 평균의 차이

In [774]:
print(df.loc[df['측정회원성별']=='M']['악력D : kg'].mean() - df.loc[df['측정회원성별']=='F']['악력D : kg'].mean())

17.55954185047464


## 남성과 여성의 교차윗몸일으키기 횟수의 평균 차이

In [776]:
print(df.loc[df['측정회원성별']=='M']['교차윗몸일으키기 : 회'].mean() - df.loc[df['측정회원성별']=='F']['교차윗몸일으키기 : 회'].mean())

14.243156833157627


# 기온 강수량 데이터

## 데이터 로드

In [777]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/weather/weather2.csv")
df.head()

,time,이화동강수,이화동기온,수영동강수,수영동기온
0,2020-01-01 00:00:00,0.1,-3.9,0.0,2.5
1,2020-01-01 01:00:00,0.0,-3.1,0.0,3.4
2,2020-01-01 02:00:00,0.0,-1.8,0.0,3.8
3,2020-01-01 03:00:00,0.0,-0.8,0.0,4.6
4,2020-01-01 04:00:00,0.0,-0.1,0.0,5.1


## 여름철(6월, 7월, 8월) 이화동이 수영동보다 높은 기온을 가진 시간대

In [788]:
df['time'] = pd.to_datetime(df['time'])
df['month'] = df['time'].dt.month
df['hour'] = df['time'].dt.hour
print(len(df.loc[((df.month == 6)|(df.month == 7)|(df.month == 8))&(df.수영동기온<df.이화동기온)]))

1415


## 이화동과 수영동의 최대강수량의 시간대

In [803]:
[df.sort_values('이화동강수',ascending=False).time.head(1)],[df.sort_values('수영동강수',ascending=False).time.head(1)]

([6561   2020-09-30 09:00:00
  Name: time, dtype: datetime64[ns]],
 [4908   2020-07-23 12:00:00
  Name: time, dtype: datetime64[ns]])

# 서비스 이탈예측 데이터

## 데이터 로드

In [804]:
import pandas as pd
#데이터 로드
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/churn/train.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,6842,15793491,Cherkasova,714,Germany,Male,26,3,119545.48,2,1,0,65482.94,0
1,8963,15607874,Keane,687,France,Male,38,0,144450.58,1,0,1,137276.83,0
2,7047,15737627,Rivero,589,Germany,Female,20,2,121093.29,2,1,0,3529.72,0
3,7503,15697844,Whitehouse,721,Spain,Female,32,10,0.00,1,1,0,136119.96,1
4,3439,15722404,Carpenter,445,France,Female,30,3,0.00,2,1,1,127939.19,0


## 남성 이탈이 가장 많은 국가 및 이탈 인원

In [811]:
print(df.loc[(df.Gender=='Male')&(df.Exited==1)].Geography.value_counts().head(1))

Geography
Germany    287
Name: count, dtype: int64


## 카드를 소유(HasCrCard==1)하고 있으면서 활성멤버(IsActiveMember==1)인 고객들의 평균 나이(소숫점 4자리)

In [814]:
print(round(df.loc[(df.HasCrCard==1)&(df.IsActiveMember==1)].Age.mean(),4))

39.6102


## Balance 값이 중간값 이상을 가지는 고객들의 CreditScore의 표준편차(소숫점 3자리

In [816]:
print(round(df.loc[df.Balance>= df.Balance.median()].CreditScore.std(),3))

97.295


# 성인 건강검진 데이터

## 데이터 로드

In [817]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/train.csv")
df.head()

,성별코드,연령대코드(5세단위),신장(5Cm단위),체중(5Kg단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),수축기혈압,...,혈색소,요단백,혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT,감마지티피,흡연상태,구강검진수검여부,치아우식증유무,치석
0,F,55,145,55,73.0,0.7,0.5,1.0,1.0,129.0,...,12.9,1.0,0.9,172.0,209.0,15.0,0,Y,0.0,Y
1,M,40,180,55,74.0,1.5,1.5,1.0,1.0,102.0,...,15.6,1.0,0.8,30.0,19.0,23.0,1,Y,0.0,Y
2,F,55,150,50,72.0,1.0,0.2,1.0,1.0,116.0,...,14.1,1.0,1.0,31.0,19.0,10.0,0,Y,0.0,N
3,M,40,170,85,89.0,1.0,0.8,1.0,1.0,124.0,...,16.7,1.0,1.0,26.0,38.0,108.0,1,Y,1.0,Y
4,F,40,155,45,62.0,0.5,1.0,1.0,1.0,127.0,...,12.9,1.0,0.7,20.0,11.0,13.0,0,Y,0.0,N


## 연령대 코드별 각 그룹 중 '혈압차'의 분산이 5번째로 큰 연령대 코드

In [824]:
df['혈압차'] = df['수축기혈압'] - df['이완기혈압']
print(df.groupby('연령대코드(5세단위)').혈압차.std().sort_values(ascending=False).index[4])

60


## WHtR 지표상 비만인 인원의 남/여 비율

In [827]:
df['WHtR'] = df['허리둘레']/df['신장(5Cm단위)']
print(len(df.loc[(df.WHtR >= 0.58)&(df.성별코드 =='M')])/len(df.loc[(df.WHtR >= 0.58)&(df.성별코드 =='F')]))

1.1693877551020408


# 자동차 보험가입 예측 데이터

## 데이터 로드

In [828]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/train.csv")
df.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,188957,Female,25,1,17.0,1,< 1 Year,No,38785.0,152.0,59,0
1,275631,Male,20,1,22.0,0,< 1 Year,No,2630.0,160.0,113,0
2,329036,Female,40,1,41.0,0,1-2 Year,Yes,33165.0,124.0,188,0
3,227288,Female,28,1,30.0,1,< 1 Year,No,29038.0,152.0,233,0
4,300441,Female,50,1,28.0,0,1-2 Year,Yes,31325.0,124.0,258,0


## Vehicle_Age 값이 2년 이상인 사람들만 필터링하고 Annual_Premium 값이 전체 데이터의 중간값 이상인 사람들의 VIntage 값 평균

In [847]:
print(df.loc[(df.Vehicle_Age == '> 2 Years')&(df.Annual_Premium >= df.Annual_Premium.median())].Vintage.mean())

154.43647182359118


## vehicle_age에 따른 각 성별 그룹의 Annual_Premium값의 평균

In [851]:
print(pd.pivot_table(df,index='Vehicle_Age',columns='Gender',values='Annual_Premium',aggfunc='mean'))

Gender             Female          Male
Vehicle_Age                            
1-2 Year     30762.245001  30413.088469
< 1 Year     29972.286702  30310.982212
> 2 Years    36108.366374  35303.870627


# 핸드폰 가격 예측 데이터

## 데이터 로드

In [852]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/mobile/train.csv")
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


## price_range의 각 value를 그룹핑하여 각 그룹의 n_cores의 빈도가 가장 높은 value와 그 빈도수

In [867]:
result = df.groupby(['price_range','n_cores']).size().reset_index()
print(result.sort_values(['price_range',0],ascending=False).groupby('price_range').head(1))

    price_range  n_cores   0
28            3        5  70
19            2        4  73
8             1        1  76
1             0        2  69


## price_range 값이 3인 그룹에서 상관관계가 2번째로 높은 두 컬럼과 그 상관계수

In [881]:
result = df.loc[df['price_range']==3].corr().unstack().reset_index().dropna()
result.loc[ result[0] !=1].sort_values(0,ascending=False).iloc[1]

level_0          fc
level_1          pc
0          0.635166
Name: 94, dtype: object

# 비행탑승 경험 만족도 데이터

## 데이터 로드

In [882]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/train.csv")
df.head()

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,55882,Male,Loyal Customer,42,Business travel,Business,473,1,1,3,...,5,5,5,5,3,5,3,3,0.0,satisfied
1,31906,Female,Loyal Customer,31,Business travel,Business,3271,4,4,4,...,3,3,5,5,3,4,3,1,5.0,satisfied
2,85481,Male,Loyal Customer,51,Personal Travel,Eco,214,5,0,5,...,3,4,4,4,4,4,3,2,0.0,satisfied
3,102134,Male,Loyal Customer,44,Business travel,Business,3097,1,1,3,...,4,4,4,4,3,4,5,0,0.0,satisfied
4,31580,Male,Loyal Customer,33,Business travel,Business,3286,1,1,1,...,4,2,5,2,5,1,4,0,0.0,satisfied


## Arrival Delay in Minutes 컬럼이 결측치인 데이터들 중 'neutral or dissatisfied''보다 satisfied'의 수가 더 높은 Class

In [889]:
result = df.loc[(df['Arrival Delay in Minutes'].isnull())].groupby(['Class','satisfaction']).size().unstack()
print(result.loc[result['satisfied'] > result['neutral or dissatisfied']].index)

Index(['Business'], dtype='object', name='Class')


# 수질 음용성 여부 데이터

## 데이터 로드

In [890]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/waters/train.csv")
df.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,7.918150,214.186611,23823.492888,7.290878,341.173322,411.424483,19.585002,25.057375,4.028958,0
1,5.422446,205.266080,18542.957451,5.491963,306.702227,382.080129,10.504023,67.493450,2.911751,1
2,7.341547,187.672402,21273.457066,7.784003,NaN,332.084293,16.842334,55.019151,4.025644,0
3,9.056245,197.666301,17403.532167,7.688917,337.460176,414.766631,15.349869,63.696746,3.319354,0
4,5.039374,142.860598,40829.353167,7.271543,NaN,386.803057,16.823773,52.297113,4.957420,0


## ph 결측치를 제외한 나머지 데이터들 중 사분위값 기준 하위 25%의 값들의 평균값

In [896]:
result = df.dropna(subset='ph')
print(result.loc[result.ph <= result.ph.describe()['25%']].ph.mean())

5.057093462441731


# 의료 비용 예측 데이터

## 데이터 로드

In [897]:
import pandas as pd
train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/MedicalCost/train.csv")
train.head()

,age,sex,bmi,children,smoker,region,charges
0,64,female,39.330,0,no,northeast,14901.51670
1,47,female,27.830,0,yes,southeast,23065.42070
2,52,female,33.300,2,no,southwest,10806.83900
3,33,female,22.135,1,no,northeast,5354.07465
4,30,male,31.400,1,no,southwest,3659.34600


## 흡연자와 비흡연자 각각 charges의 상위 10% 그룹의 평균 차이

In [911]:
print(train.loc[(train.smoker=='yes')&(train.charges >= train.loc[train.smoker=='yes'].charges.sort_values(ascending=False).iloc[int(len(train.loc[train.smoker=='yes'])*0.1)-1])].charges.mean() - train.loc[(train.smoker=='no')&(train.charges >= train.loc[train.smoker=='no'].charges.sort_values(ascending=False).iloc[int(len(train.loc[train.smoker=='no'])*0.1)-1])].charges.mean())

29297.954548156144


# 킹카운티 주거지 가격예측문제 데이터

## 데이터 로드

In [912]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/kingcountyprice//train.csv")
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,8961990160,20150413T000000,567500.0,3,2.5,2080,4556,2.0,0,0,...,8,2080,0,1999,0,98074,47.6036,-122.014,1530,5606
1,9455200205,20140604T000000,525000.0,3,2.0,1540,7800,1.0,0,0,...,8,1540,0,2004,0,98125,47.7041,-122.288,1510,7800
2,7853220670,20140918T000000,540000.0,3,2.5,2860,8935,2.0,0,0,...,8,2860,0,2004,0,98065,47.5336,-121.855,2650,6167
3,3298201170,20141110T000000,350000.0,3,1.0,940,7811,1.0,0,0,...,6,940,0,1959,0,98008,47.6195,-122.118,1180,7490
4,7972604355,20140521T000000,218000.0,3,1.0,1020,7874,1.0,0,0,...,7,1020,0,1956,0,98106,47.5175,-122.346,1290,7320


## bedrooms의 빈도가 가장 높은 값을 가지는 데이터들의 price의 상위 10%와 하위 10% 값의 차이

In [934]:
print(df.loc[df.bedrooms == df.bedrooms.value_counts().index[0]].price.quantile(0.9)- df.loc[df.bedrooms == df.bedrooms.value_counts().index[0]].price.quantile(0.1))

505500.0


# 대학원 입학가능성 데이터

## 데이터 로드

In [935]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/admission/train.csv")
df.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,246,328,110,4,4.0,2.5,9.02,1,0.81
1,100,323,113,3,4.0,4.0,8.88,1,0.79
2,79,296,95,2,3.0,2.0,7.54,1,0.44
3,53,334,116,4,4.0,3.0,8.00,1,0.78
4,444,321,114,5,4.5,4.5,9.16,1,0.87


## Serial No.컬럼을 제외하고 'Chance of Admit'을 종속변수, 나머지 변수를 독립변수라 할 때, 랜덤포레스트를 통해 회귀 예측을 할 때 변수중요도 값

In [960]:

target = df['Chance of Admit']
train = df.drop(['Serial No.','Chance of Admit'],axis= 1)
from sklearn.ensemble import RandomForestRegressor
train
rf = RandomForestRegressor()
rf.fit(train,target)
result = pd.DataFrame({'importance':rf.feature_importances_},train.columns)
print(result.sort_values('importance',ascending=False))

                   importance
CGPA                 0.813163
GRE Score            0.087473
TOEFL Score          0.029226
SOP                  0.025182
LOR                  0.023677
University Rating    0.013544
Research             0.007734


# 레드 와인 퀄리티 예측 데이터

## 데이터 로드

In [977]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/redwine/train.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,5.0,0.38,0.01,1.6,0.048,26.0,60.0,0.99084,3.70,0.75,14.0,6
1,5.0,0.42,0.24,2.0,0.060,19.0,50.0,0.99170,3.72,0.74,14.0,8
2,7.1,0.36,0.30,1.6,0.080,35.0,70.0,0.99693,3.44,0.50,9.4,5
3,7.6,0.29,0.49,2.7,0.092,25.0,60.0,0.99710,3.31,0.61,10.1,6
4,7.7,0.51,0.28,2.1,0.087,23.0,54.0,0.99800,3.42,0.74,9.2,5


## quality 값이 3인 그룹과 8인 데이터그룹의 각 컬럼별 독립변수의 표준편차 값의 차이가 가장 큰 컬럼명

In [978]:
print((abs(df.loc[df.quality == 3].std() - df.loc[df.quality == 8].std())).sort_values(ascending=False).index[0])

total sulfur dioxide


# 약물 분류 데이터

## 데이터 로드

In [969]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/drug/train.csv")
df.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,20,F,NORMAL,NORMAL,9.281,4
1,24,M,HIGH,NORMAL,9.475,1
2,34,M,NORMAL,HIGH,22.456,0
3,17,M,NORMAL,NORMAL,10.832,4
4,72,M,LOW,HIGH,16.310,0


## 남성들의 연령대별 Na_to_K값의 평균값

In [975]:
df['Age2'] = df.Age.map(lambda x : (x//10)*10)
print(df.loc[df.Sex=='M'].groupby('Age2').Na_to_K.mean().to_frame())

        Na_to_K
Age2           
10    13.627000
20    18.260769
30    15.244143
40    13.897273
50    14.811273
60    14.284308
70    11.363857


# 사기회사 분류 데이터

## 데이터 로드

In [979]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/train.csv")
df.head()

,Sector_score,LOCATION_ID,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
0,2.72,14,4.53,0.6,2.718,87.93,0.6,52.758,92.46,5.0,...,0.4,0,0.2,0.0,4.0,108.362,0.4,0.5,21.6724,1
1,3.41,16,12.68,0.6,7.608,41.00,0.6,24.600,53.68,5.5,...,0.4,0,0.2,0.0,4.4,41.936,0.4,0.5,8.3872,1
2,2.72,11,3.11,0.6,1.866,113.97,0.6,68.382,117.08,5.5,...,0.4,0,0.2,0.0,4.4,88.832,0.4,0.5,17.7664,1
3,3.41,8,1.12,0.4,0.448,0.00,0.2,0.000,1.12,5.0,...,0.4,0,0.2,0.0,2.2,1.848,0.4,0.5,0.3696,0
4,55.57,9,1.06,0.4,0.424,0.00,0.2,0.000,1.06,5.0,...,0.4,0,0.2,0.0,2.2,1.824,0.4,0.5,0.3648,0


## 데이터의 Risk 값에 따른 score_a와 score_b 평균값

In [985]:
print(df.groupby('Risk')[['Score_A','Score_B']].mean())

       Score_A   Score_B
Risk                    
0     0.262234  0.225532
1     0.490164  0.444262


# 센서 데이터 동작 유형 분류 데이터

## 데이터 로드

In [986]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/train.csv")
df.head()

,motion_0,motion_1,motion_2,motion_3,motion_4,motion_5,motion_6,motion_7,motion_8,motion_9,...,motion_55,motion_56,motion_57,motion_58,motion_59,motion_60,motion_61,motion_62,motion_63,pose
0,-6.0,8.0,5.0,6.0,-12.0,-27.0,5.0,24.0,-8.0,-5.0,...,-1.0,-3.0,-6.0,-9.0,-7.0,36.0,68.0,3.0,-3.0,1
1,-32.0,-3.0,-4.0,-5.0,-1.0,16.0,0.0,-5.0,42.0,2.0,...,-17.0,-12.0,-1.0,-1.0,0.0,-26.0,-11.0,-4.0,-9.0,1
2,35.0,-2.0,-1.0,1.0,-25.0,-16.0,-3.0,-20.0,10.0,1.0,...,6.0,-65.0,6.0,-10.0,-31.0,-19.0,-3.0,-47.0,-17.0,0
3,-15.0,-6.0,-3.0,-5.0,27.0,37.0,-2.0,-2.0,19.0,5.0,...,2.0,11.0,0.0,-2.0,-4.0,-7.0,15.0,3.0,14.0,1
4,60.0,3.0,0.0,-7.0,-6.0,-10.0,-6.0,7.0,-11.0,-4.0,...,-12.0,17.0,-3.0,1.0,2.0,-2.0,20.0,18.0,10.0,0


## pose값에 따른 각 motion 컬럼의 중간값의 가장 큰 차이를 보이는 motion 컬럼 및 값

In [1009]:
result = df.groupby('pose').median().T
result['diff'] = abs(result[0] - result[1])
print(result[result['diff'] == max(result['diff'])].index)

Index(['motion_54', 'motion_62'], dtype='object')


# 현대 차량 가격 분류 문제 데이터

## 데이터 로드

In [1010]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/hyundai/train.csv")
df.head()

,model,year,price,transmission,mileage,fuelType,tax(£),mpg,engineSize
0,Tucson,2016,12795,Manual,36263,Diesel,30,61.7,1.7
1,I10,2012,3995,Manual,22089,Petrol,20,61.4,1.2
2,I30,2019,24000,Manual,555,Petrol,145,34.0,2.0
3,Tucson,2017,12995,Manual,32147,Diesel,30,61.7,1.7
4,Tucson,2018,14991,Semi-Auto,32217,Diesel,150,57.6,1.7


## 정보가 가장 많은 상위 3차종의 price 값의 각 평균값

In [1015]:
df.loc[df.model.isin(df.groupby('model').size().sort_values(ascending=False).head(3).index)].groupby('model').price.mean()

model
I10        7646.137891
I30       11651.821759
Tucson    15805.441373
Name: price, dtype: float64

# 당뇨여부 판단 데이터

## 데이터 로드

In [1016]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/diabetes/train.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,3,102,74,0,0,29.5,0.121,32,0
1,2,144,58,33,135,31.6,0.422,25,1
2,5,136,82,0,0,0.0,0.640,69,0
3,13,145,82,19,110,22.2,0.245,57,0
4,1,117,60,23,106,33.8,0.466,27,0


## outcome 값에 따른 각 그룹의 각 컬럼의 평균 차이

In [1021]:
result = df.groupby('Outcome').mean().T
result['diff'] = abs(result[0]-result[1])
print(result['diff'])

Pregnancies                  1.574159
Glucose                     29.943995
BloodPressure                3.202079
SkinThickness                2.952033
Insulin                     33.398645
BMI                          4.541437
DiabetesPedigreeFunction     0.143646
Age                          5.866939
Name: diff, dtype: float64


# 넷플릭스 주식 데이터

## 데이터 로드

In [1022]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/nflx/NFLX.csv")
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2002-05-23,1.156429,1.242857,1.145714,1.196429,1.196429,104790000
1,2002-05-24,1.214286,1.225000,1.197143,1.210000,1.210000,11104800
2,2002-05-28,1.213571,1.232143,1.157143,1.157143,1.157143,6609400
3,2002-05-29,1.164286,1.164286,1.085714,1.103571,1.103571,6757800
4,2002-05-30,1.107857,1.107857,1.071429,1.071429,1.071429,10154200


## 매년 5월달의 open 가격의 평균값

In [1028]:
df.Date = pd.to_datetime(df.Date)
df.loc[df.Date.dt.month == 5].groupby(df.Date.dt.to_period('M')).Open.mean()

Date
2002-05      1.155833
2003-05      1.641497
2004-05      4.261143
2005-05      1.951905
2006-05      4.163571
2007-05      3.159351
2008-05      4.435034
2009-05      5.790571
2010-05     14.417071
2011-05     34.650272
2012-05     10.523247
2013-05     31.936429
2014-05     51.121292
2015-05     85.057429
2016-05     92.705715
2017-05    158.255455
2018-05    329.779541
2019-05    359.664548
2020-05    433.880499
2021-05    496.923996
Freq: M, Name: Open, dtype: float64

# nba 선수 능력치 데이터

## 데이터 로드

In [1029]:
pd.set_option('display.max_columns',50)
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/nba/nba.csv",encoding='latin',sep=';')
df.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Precious Achiuwa,C,22,TOR,73,28,23.6,3.6,8.3,0.439,0.8,2.1,0.359,2.9,6.1,0.468,0.486,1.1,1.8,0.595,2.0,4.5,6.5,1.1,0.5,0.6,1.2,2.1,9.1
1,2,Steven Adams,C,28,MEM,76,75,26.3,2.8,5.1,0.547,0.0,0.0,0.000,2.8,5.0,0.548,0.547,1.4,2.6,0.543,4.6,5.4,10.0,3.4,0.9,0.8,1.5,2.0,6.9
2,3,Bam Adebayo,C,24,MIA,56,56,32.6,7.3,13.0,0.557,0.0,0.1,0.000,7.3,12.9,0.562,0.557,4.6,6.1,0.753,2.4,7.6,10.1,3.4,1.4,0.8,2.6,3.1,19.1
3,4,Santi Aldama,PF,21,MEM,32,0,11.3,1.7,4.1,0.402,0.2,1.5,0.125,1.5,2.6,0.560,0.424,0.6,1.0,0.625,1.0,1.7,2.7,0.7,0.2,0.3,0.5,1.1,4.1
4,5,LaMarcus Aldridge,C,36,BRK,47,12,22.3,5.4,9.7,0.550,0.3,1.0,0.304,5.1,8.8,0.578,0.566,1.9,2.2,0.873,1.6,3.9,5.5,0.9,0.3,1.0,0.9,1.7,12.9


## Tm 컬럼은 각 팀의 의미함, TOR 팀의 평균나이(소수 4째자리)

In [1032]:
print(round(df.loc[df.Tm=='TOR'].Age.mean(),4))

24.8696


## 전체 선수 중 중 최소 나이대의 선수들을 필터하고 그들 중 가장 많은 포지션 출력

In [1037]:
print(df.loc[df.Age==df.Age.min()].Pos.value_counts().index[0])

SG


## 선수들의 이름은 first_name + 공백 + last_name으로 이루어져 있음. 가장 많은 first_name

In [1043]:
df['first_name'] = df['Player'].map(lambda x : x.split()[0])
print(df.first_name.value_counts().index[0])

Justin


## PTS 컬럼은 경기당 평균 득점수, 각 포지션별로 경기당 평균득점수의 평균

In [1045]:
print(df.groupby('Pos').PTS.mean())

Pos
C         7.690769
C-PF     12.850000
PF        7.737500
PF-SF     8.500000
PG        8.325161
PG-SG    22.000000
SF        7.270253
SF-SG     8.660000
SG        7.810553
SG-PG     9.525000
SG-SF     7.360000
Name: PTS, dtype: float64


## G컬럼은 참여한 경기의 숫자, 각 팀별로 가장 높은 경기참여 수를 가진 선수들의 경기 참여 숫자의 평균

In [1055]:
print(df.sort_values(['Tm','G'],ascending=False).groupby('Tm').head(1).G.mean())

77.51612903225806


## Tm의 값이 MIA이며 Pos는 C 또는 PF인 선수의 MP값의 평균

In [1057]:
print(df.loc[(df.Tm=='MIA')&((df.Pos == 'C')|(df.Pos=='PF'))].MP.mean())

16.7875


## 전체 데이터 중 G의 평균값의 1.5배 이상이 데이터들만 추출했을 때 3P값의 평균

In [1061]:
print(df.loc[df.G >= df.G.mean()*1.5]['3P'].mean())

1.3853658536585365


## Age의 평균 이상인 그룹과 평균 미만인 그룹간의 G값의 평균의 차이

In [1063]:
print(df.loc[df.Age >= df.Age.mean()].G.mean() - df.loc[df.Age < df.Age.mean()].G.mean())

3.787674551781862


## 평균나이가 가장 젊은 팀

In [1067]:
print(df.groupby('Tm').Age.mean().sort_values().index[0])

MEM


## Pos 그룹별 평균 MP 값

In [1069]:
print(df.groupby('Pos').MP.mean())

Pos
C        16.990000
C-PF     25.350000
PF       17.937500
PF-SF    27.300000
PG       19.547742
PG-SG    37.200000
SF       17.514557
SF-SG    20.340000
SG       18.554271
SG-PG    22.950000
SG-SF    19.620000
Name: MP, dtype: float64
